#### - replace gait periodes by single steps (step detection von elisa, alt.: end_index detection)
#### - only positive angles!
#### - implement PSTHs for parameters
#### - implement max_value for parameters
#### - export function
####
####
#### - Kamera Kalibrierung aufnehmen
#### - Outlier Frames 200x, Re-Trainieren
#### - GaitAnalysis aus Paper?

# Walkthrough 2D

In [ ]:
from Analysis.gaitanalysis import Recording2D
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
bot = Recording2D(filepath="/Users/kobel/Documents/Medizin/Doktorarbeit/Coding/Inken Analyse/Dummy Data Inken/2-1413_Catwalk_Maus01_prä-OP1_20220315DLC_resnet50_2-1413 CatwalkMay25shuffle1_600000.csv", recorded_framerate=30)
bot.metadata

In [ ]:
bot.run()#video_filepath='/Users/kobel/Documents/Medizin/Doktorarbeit/Coding/Inken Analyse/Dummy Data Inken/2-1413_Catwalk_Maus01_prä-OP1_20220315.mp4'
bot.bodyparts['Snout'].df.head()

In [ ]:
bot.run_gait_analysis()

In [ ]:
#to implement
from statistics import mean
psth = bot.parameters_over_steps['angle_paw_knee_centerofgravity_right_HindPawRight'].mean(axis = 0)
max_ = bot.parameters_over_steps['angle_paw_knee_centerofgravity_right_HindPawRight'].max(axis = 1)
mean_max = mean(max_)

In [ ]:
plt.close()
fig, ax = plt.subplots()
plt.plot(range(-7, 7), psth) 
plt.show()

## Proof undistort points

In [ ]:
import imageio.v4 as iio
import cv2
import pickle


In [ ]:
df = pd.read_hdf("196_F7-27_220826_OTT_Bottom_synchronizedDLC_resnet152_OT_BottomCam_finalSep20shuffle1_550000filtered.h5")
df = df.drop('scorer', axis=1)
df.columns = df.iloc[0, :]+ '_' + df.iloc[1, :]
df = df.drop([0, 1], axis=0)
df = df.reset_index()
df = df.drop('index', axis=1)
df = df.astype(float)

In [ ]:
def load_intrinsic_camera_calibration(filepath_intrinsic_calibration: Path, xy_offset: Tuple[int, int]) -> None:
    with open(filepath_intrinsic_calibration, 'rb') as io:
        intrinsic_calibration = pickle.load(io)
    adjusted_intrinsic_calibration = intrinsic_calibration.copy()
    intrinsic_calibration_video_size = intrinsic_calibration['size']
    x_offset, y_offset = xy_offset[0], xy_offset[1]
    adjusted_K = intrinsic_calibration['K'].copy()
    adjusted_K[0][2] = adjusted_K[0][2] - x_offset
    adjusted_K[1][2] = adjusted_K[1][2] - y_offset
    return adjusted_K, intrinsic_calibration['D']

In [ ]:
image=iio.imread('/Users/kobel/Downloads/196_F7-27_220826_OTT_Bottom_synchronizedDLC_resnet152_OT_BottomCam_finalSep20shuffle1_550000_filtered_labeled.mp4', index=5700)

In [ ]:
size = (image.shape[1], image.shape[0])

In [ ]:
K, D = load_intrinsic_camera_calibration('/Users/kobel/Downloads/OneDrive_3_30/Bottom_checkerboard_intrinsic_calibration_results.p', xy_offset=(136, 452))

In [ ]:
dst_list = []
for bp in ['Snout', 'TailBase', 'HindPawLeft', 'HindPawRight', 'ForePawLeft', 'ForePawRight', 'WristLeft', 'WristRight', 'TailQuarterAnt', 'TaiMiddle', 'TailQuarterPost', 'TailEnd']:
    pt = df[[bp + '_x', bp + '_y']].copy().values
    newcameramtx, _ = cv2.getOptimalNewCameraMatrix(K, D, size, 1, size)
    dst = cv2.undistortPoints(pt, K, D, None, newcameramtx)
    dst = np.squeeze(dst)
    dst_list.append(dst)

In [ ]:
plt.close()
fig, ax = plt.subplots()
ax.set_aspect('equal')
#plt.imshow(undistort_fisheye_image_for_inspection(image=image, K=K, D=D, size=size))
# issue: image is rectified, points are not -> plotting points over image makes no sense
for dst in dst_list:
    plt.scatter(dst[5700][0], dst[5700][1])
plt.show()